In [46]:
import re
import pandas as pd
from datetime import datetime 

fecha = datetime.today().strftime('%Y%m%d')
num_format = re.compile("^[\-]?[0-9]*\.?[0-9]+$")

In [31]:
df_arveja = pd.read_csv("D:\AEPS 2.0 Boyaca\datos\CULTIVOS-principal\Datos\_Arveja.csv", encoding='ISO-8859-1', thousands=',')
df_papa = pd.read_csv("D:\AEPS 2.0 Boyaca\datos\CULTIVOS-principal\Datos\_Papa.csv", encoding='ISO-8859-1', thousands=',')
df_cebolla = pd.read_csv("D:\AEPS 2.0 Boyaca\datos\CULTIVOS-principal\Datos\_Cebolla.csv", encoding='ISO-8859-1', thousands=',')
df_zanahoria = pd.read_csv("D:\AEPS 2.0 Boyaca\datos\CULTIVOS-principal\Datos\_Zanahoria.csv", encoding='ISO-8859-1', thousands=',')

In [32]:
df_papa['CULT'] = 'Papa'
df_arveja['CULT'] = 'Arveja'
df_cebolla['CULT'] = 'Cebolla'
completo = pd.DataFrame()

completo = completo.append(df_papa, ignore_index=True)
completo = completo.append(df_arveja, ignore_index=True)
completo = completo.append(df_zanahoria, ignore_index=True)
completo = completo.append(df_cebolla, ignore_index=True)

completo['L1_0'] = completo['L1_0'].apply( lambda x: x[0:9]) #Se eliminan letras al final del codigo agricultor



In [34]:
#agregar consecutivos en los datos repetidos de la serie para diferenciar
def consecutive(x, serie):
    conse = 1
    for j in x:
        if conse != 1 :
            serie[j] = serie[j] + f'-{conse}'
            print(f'Dato modificado en el indice {j} con: ' + ext_id_event[j])
        conse += 1

In [35]:
def duplicado_consecutive(serie):
    duplic = serie[serie.duplicated(keep=False)] #Se verifica si hay duplicados en la serie

    if not duplic.empty :
        duplic = pd.DataFrame(duplic)

        duplic = duplic.groupby(duplic.columns.tolist()).apply(lambda x: x.index.tolist())
        
        duplic.apply(consecutive, args=(serie,))
    return serie

In [47]:
#far_production_events
# 'technical'	'ext_id'	'plot'	'form'

ext_id_plot = 'LOTE-' + completo['L1_0'] #Se genera codigo del lote con el codigo del agricultor

ext_id_event = 'EVENTO-CUL-' + fecha + '-' + completo['L1_0'] #Se genera los id externos para los eventos productivos

ext_id_event = duplicado_consecutive(ext_id_event) #Se agrega un consecutivo a los id duplicados para diferenciar los eventos

completo['event'] = ext_id_event #Se agregan los id de evento para futuras implementaciones

e = {'technical' : 1, 'ext_id' : ext_id_event, 'plot': ext_id_plot, 'form' : 3}

event = pd.DataFrame(e)

event.to_csv('far_production_events.csv', index=False)

Dato modificado en el indice 544 con: EVENTO-CUL-20211202-MO-CA-001-2
Dato modificado en el indice 416 con: EVENTO-CUL-20211202-MO-CA-002-2
Dato modificado en el indice 545 con: EVENTO-CUL-20211202-MO-CA-002-3
Dato modificado en el indice 603 con: EVENTO-CUL-20211202-MO-CA-002-4
Dato modificado en el indice 430 con: EVENTO-CUL-20211202-MO-CA-011-2
Dato modificado en el indice 604 con: EVENTO-CUL-20211202-MO-CA-011-3
Dato modificado en el indice 421 con: EVENTO-CUL-20211202-MO-CA-016-2
Dato modificado en el indice 547 con: EVENTO-CUL-20211202-MO-CA-016-3
Dato modificado en el indice 549 con: EVENTO-CUL-20211202-MO-CA-018-2
Dato modificado en el indice 411 con: EVENTO-CUL-20211202-MO-CA-019-2
Dato modificado en el indice 601 con: EVENTO-CUL-20211202-MO-CA-019-3
Dato modificado en el indice 423 con: EVENTO-CUL-20211202-MO-CA-020-2
Dato modificado en el indice 413 con: EVENTO-CUL-20211202-MO-CA-025-2
Dato modificado en el indice 602 con: EVENTO-CUL-20211202-MO-CA-025-3
Dato modificado en e

In [48]:
def question (frame, event, question, value, tipo=''):
    #valida si tiene un tipo y si el valor es diferente de nan
    if tipo and value == value:
        #se verifica si es numero de no serlo se deja en cero
        isnumber = re.match(num_format,str(value)) 
        if not isnumber:
            value = 0

    frame = frame.append({'event': event, 'raw_value': value, 'question': question, 'fixed_value': value,'validated': 1}, ignore_index=True)
    return frame

In [49]:
far_responses_text = pd.DataFrame(columns=['event','raw_value','question','fixed_value','validated'])
far_responses_numeric = pd.DataFrame(columns=['event','raw_value','question','fixed_value','raw_units','fixed_units','validated'])

for index, row in completo.iterrows():

    print('Generando Preguntas del evento ' + row['event'] )

    #Cultivos

    far_responses_text = question(far_responses_text, row['event'], "CULT", row['CULT'])
    far_responses_text = question(far_responses_text, row['event'], "VAR", row['VAR'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "ARV_1", row['ARV_1'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "ARV_2", row['ARV_2'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "ARV_3", row['ARV_3'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "ARV_4", row['ARV_4'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "ARV_5", row['ARV_5'])
    far_responses_text = question(far_responses_text, row['event'], "ARV_6", row['ARV_6'])
    far_responses_text = question(far_responses_text, row['event'], "ARV_7", row['ARV_7'])
    far_responses_text = question(far_responses_text, row['event'], "ARV_8", row['ARV_8'])
    far_responses_text = question(far_responses_text, row['event'], "ARV_9", row['ARV_9'])
    far_responses_text = question(far_responses_text, row['event'], "ARV_10", row['ARV_10'])
    far_responses_text = question(far_responses_text, row['event'], "CEB_1", row['CEB_1'])
    far_responses_text = question(far_responses_text, row['event'], "CEB_2", row['CEB_2'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "CEB_3", row['CEB_3'])
    far_responses_numeric = question(far_responses_numeric, row['event'], "REND", row['REND'])


far_responses_text.to_csv('far_responses_text.csv', index=False)
far_responses_numeric.to_csv('far_responses_numeric.csv', index=False)

Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-005
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-005-2
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-005-3
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-005-4
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-002
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-002-2
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-002-3
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-008
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-008-2
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-008-3
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-008-4
Generando Preguntas del evento EVENTO-CUL-20211202-SI-FI-001
Generando Preguntas del evento EVENTO-CUL-20211202-SI-FI-001-2
Generando Preguntas del evento EVENTO-CUL-20211202-SI-FI-001-3
Generando Preguntas del evento EVENTO-CUL-20211202-SI-SA-001
Generando Preguntas del evento EVENTO-CUL-20211202-SI-TO-003
Gene